In [10]:
import numpy as np
path_img = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/brats2024/brats-GoAT/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/BraTS-GoAT-00000/BraTS-GoAT-00000-seg.nii.gz"

import nibabel as nib 

# load the image
img = nib.load(path_img)

# print shape of the image
print(img.shape)
# print all the classes in the image
print(np.unique(img.get_fdata()))

# get the dtype
print(img.get_fdata().dtype)

(240, 240, 155)
[0. 1. 2. 3.]
float64


In [3]:
import os
import json


base_path = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/brats2024/brats-GoAT/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth"
output_json_path = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/brats2024/brats-GoAT/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/brats2024.json"

import os
import json
import random

def create_json(base_path, output_json_path, train_proportion=0.8):
    """
    Create a JSON file with the structure required for the training and validation data.
    
    Args:
        base_path (str): The base directory containing the data folders.
        output_json_path (str): The path where the JSON file will be saved.
        train_proportion (float): Proportion of data to be used for training (between 0 and 1).
    
    Returns:
        None
    """
    data_entries = []
    
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        
        if os.path.isdir(folder_path):
            # Find all the necessary files in the folder
            image_files = []
            label_file = None
            
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if "seg" in file_name and file_name.endswith(".nii.gz"):
                    label_file = os.path.join(folder_name, file_name)  # Relative path
                elif file_name.endswith(".nii.gz"):
                    image_files.append(os.path.join(folder_name, file_name))  # Relative path
            
            # Sort the image files to have a consistent order (optional but recommended)
            image_files.sort()
            
            if label_file is not None and image_files:
                data_entries.append({
                    "image": image_files,
                    "label": label_file
                })
    
    # Shuffle the data to ensure randomness
    random.shuffle(data_entries)
    
    # Split the data into training and validation sets
    split_index = int(len(data_entries) * train_proportion)
    training_data = data_entries[:split_index]
    validation_data = data_entries[split_index:]
    
    json_data = {
        "training": training_data,
        "validation": validation_data
    }
    
    with open(output_json_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)
    
    # Print the path of the created JSON file
    print(f"JSON file created at {output_json_path}")






In [6]:
base_path = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/brats2024/brats-GoAT/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth"
output_json_path = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/brats2024/brats-GoAT/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/brats2024.json"

# Create the JSON file with the dataset information
create_json(base_path, output_json_path, train_proportion=0.95)


# now read the json file again and count how many files are for training and for validation

with open(output_json_path, 'r') as json_file:
    data = json.load(json_file)

training_data = data["training"]
validation_data = data["validation"]

print(f"Number of training samples: {len(training_data)}")
print(f"Number of validation samples: {len(validation_data)}")


JSON file created at /home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/brats2024/brats-GoAT/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/brats2024.json
Number of training samples: 1283
Number of validation samples: 68


In [1]:
import os
import shutil

def delete_recursively(path_to_clean):
    """
    Deletes all files and folders inside a given directory recursively.
    
    Parameters:
    - path_to_clean: str, the path to the directory to clean.
    """
    try:
        # Check if the path exists
        if os.path.exists(path_to_clean):
            # Iterate through all files and directories in the given path
            for item in os.listdir(path_to_clean):
                item_path = os.path.join(path_to_clean, item)
                # Check if it's a directory or a file and delete accordingly
                if os.path.isdir(item_path):
                    shutil.rmtree(item_path)  # Recursively delete a directory
                else:
                    os.remove(item_path)  # Delete a file
            print(f"All contents of {path_to_clean} have been deleted successfully.")
        else:
            print(f"The path {path_to_clean} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the path
path_to_clean = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/datasets_for_ssl/brats2024_sample"

# Call the function
delete_recursively(path_to_clean)


All contents of /home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/datasets_for_ssl/brats2024_sample have been deleted successfully.


In [2]:
import os
import nibabel as nib
import shutil

def process_individual_patient(patient_path):
    """
    Process an individual patient's folder to create a dictionary where each key corresponds to a modality 
    (seg, t1c, t1n, t2f, t2w) and each value is the full path to the corresponding file.
    
    The code assumes a filename pattern like: BraTS-GLI-02405-100-t1c.nii.gz
    and extracts the modality key from the part after the last dash and before the extension.
    """
    
    file_list = os.listdir(patient_path)
    modality_dict = {}
    
    for file_name in file_list:
        if file_name.endswith(".nii.gz"):
            # Example pattern: BraTS-GLI-02405-100-t1c.nii.gz
            # Split by '-' and take the last part: t1c.nii.gz -> modality_key = t1c
            try:
                modality_key = file_name.split('-')[-1].split('.')[0]
            except IndexError:
                continue
            file_path = os.path.join(patient_path, file_name)
            modality_dict[modality_key] = file_path
    
    return modality_dict


def create_patient_modality_dict(base_path):
    """
    Create a dictionary for each patient folder, where each key corresponds to a modality (seg, t1c, t1n, t2f, t2w) 
    and each value is the full path to the corresponding file.
    """
    patient_dict = {}
    list_patient_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    
    # Sort patient folders to ensure consistent indexing
    list_patient_folders.sort()
    
    for patient_folder in list_patient_folders:
        patient_path = os.path.join(base_path, patient_folder)
        modality_dict = process_individual_patient(patient_path)
        if modality_dict:
            patient_dict[patient_folder] = modality_dict
    return patient_dict


# Paths
path_sample_brats2024 = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/brats2024/brats-GLI/training_data_additional"
path_to_copy_processed_data = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/datasets_for_ssl/brats2024_sample"

# Create mapping from original modality keys to the new naming convention
modality_name_map = {
    "seg": "seg",
    "t1c": "t1ce",
    "t1n": "t1",    # assuming t1n stands for T1 native
    "t2f": "flair", # assuming t2f stands for T2-FLAIR
    "t2w": "t2"     # assuming t2w stands for T2-weighted
}

# Create dictionary of patients and modalities
patient_modality_dict = create_patient_modality_dict(path_sample_brats2024)

# Ensure the output directory exists
os.makedirs(path_to_copy_processed_data, exist_ok=True)

# Enumerate over patients to assign new IDs in zero-based indexing
patients_sorted = sorted(patient_modality_dict.keys())

for idx, patient_folder in enumerate(patients_sorted):
    modalities = patient_modality_dict[patient_folder]
    
    # Format patient ID as 5 digits
    patient_id_str = f"{idx:05d}"  # zero-padded, 5 digits
    
    # Create patient folder name
    new_patient_name = f"BraTS2024_{patient_id_str}"
    
    # Create patient folder in the new location
    new_patient_dir = os.path.join(path_to_copy_processed_data, new_patient_name)
    os.makedirs(new_patient_dir, exist_ok=True)
    
    # Go through each modality file and rename according to the mapping
    for original_modality_key, original_file_path in modalities.items():
        if original_modality_key in modality_name_map:
            new_modality_name = modality_name_map[original_modality_key]
        else:
            continue  # skip unknown modalities
        
        # Construct new file name, e.g. BraTS2024_00000_flair.nii.gz
        new_file_name = f"{new_patient_name}_{new_modality_name}.nii.gz"
        new_file_path = os.path.join(new_patient_dir, new_file_name)
        
        # Load the image with nibabel and save to the new path
        img = nib.load(original_file_path)
        nib.save(img, new_file_path)

# Print a summary of the first few patients to confirm the output
for i, patient in enumerate(patients_sorted[:5]):
    patient_id_str = f"{i:05d}"
    new_patient_name = f"BraTS2024_{patient_id_str}"
    print(f"Patient (original): {patient} -> (new): {new_patient_name}")
    new_path = os.path.join(path_to_copy_processed_data, new_patient_name)
    new_files = os.listdir(new_path)
    for f in new_files:
        print(f"  {f}")
    print("\n")


Patient (original): BraTS-GLI-02405-100 -> (new): BraTS2024_00000
  BraTS2024_00000_t1ce.nii.gz
  BraTS2024_00000_t2.nii.gz
  BraTS2024_00000_seg.nii.gz
  BraTS2024_00000_t1.nii.gz
  BraTS2024_00000_flair.nii.gz


Patient (original): BraTS-GLI-02405-101 -> (new): BraTS2024_00001
  BraTS2024_00001_t2.nii.gz
  BraTS2024_00001_t1ce.nii.gz
  BraTS2024_00001_flair.nii.gz
  BraTS2024_00001_seg.nii.gz
  BraTS2024_00001_t1.nii.gz


Patient (original): BraTS-GLI-02406-100 -> (new): BraTS2024_00002
  BraTS2024_00002_seg.nii.gz
  BraTS2024_00002_t2.nii.gz
  BraTS2024_00002_t1.nii.gz
  BraTS2024_00002_t1ce.nii.gz
  BraTS2024_00002_flair.nii.gz


Patient (original): BraTS-GLI-02407-100 -> (new): BraTS2024_00003
  BraTS2024_00003_seg.nii.gz
  BraTS2024_00003_t2.nii.gz
  BraTS2024_00003_flair.nii.gz
  BraTS2024_00003_t1ce.nii.gz
  BraTS2024_00003_t1.nii.gz


Patient (original): BraTS-GLI-02408-100 -> (new): BraTS2024_00004
  BraTS2024_00004_t1.nii.gz
  BraTS2024_00004_t2.nii.gz
  BraTS2024_00004_seg.

# Check the dimensions of the dataset

In [5]:
path_brats2024_sample = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/datasets_for_ssl/brats2024_sample"

import os
import nibabel as nib

def process_patient_images(folder_path):
    """
    Process all patient folders within the given folder path.

    Parameters:
    folder_path (str): Path to the folder containing patient folders.

    Prints:
    The shapes of each .nii.gz file in each patient folder, formatted for readability.
    """
    if not os.path.exists(folder_path):
        print(f"The specified folder path '{folder_path}' does not exist.")
        return

    patient_folders = [
        os.path.join(folder_path, patient_folder)
        for patient_folder in os.listdir(folder_path)
        if os.path.isdir(os.path.join(folder_path, patient_folder))
    ]

    for patient_folder in patient_folders:
        print(f"Processing patient: {os.path.basename(patient_folder)}")
        nii_files = [
            os.path.join(patient_folder, file)
            for file in os.listdir(patient_folder)
            if file.endswith(".nii.gz")
        ]

        if not nii_files:
            print("No .nii.gz files found for this patient.")
            print("\n")
            continue

        for nii_file in nii_files:
            try:
                img = nib.load(nii_file)
                print(f"File: {os.path.basename(nii_file)} | Shape: {img.shape}")
            except Exception as e:
                print(f"Error loading {nii_file}: {e}")

        print("\n")  # Add space between patients for readability

# Example usage:
process_patient_images(path_brats2024_sample)

Processing patient: BraTS2024_00078
File: BraTS2024_00078_seg.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00078_t1.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00078_t2.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00078_flair.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00078_t1ce.nii.gz | Shape: (182, 218, 182)


Processing patient: BraTS2024_00044
File: BraTS2024_00044_flair.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00044_t1ce.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00044_t1.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00044_seg.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00044_t2.nii.gz | Shape: (182, 218, 182)


Processing patient: BraTS2024_00042
File: BraTS2024_00042_t1ce.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00042_t1.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00042_seg.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00042_flair.nii.gz | Shape: (182, 218, 182)
File: BraTS2024_00042_t2.nii.gz | Shape: (182, 218, 182)


Processing pa

# Check the number of classes in the dataset

(182, 218, 182)

SyntaxError: invalid syntax (<ipython-input-15-e91212c3b162>, line 4)